# **MACHINE LEARNING DE PELÍCULAS**

**OBJETIVO** Entrenar un modelo de machine learning para armar un sistema de recomendación de películas para usuarios, donde dado un id de usuario y una película, nos diga si la recomienda o no para dicho usuario.

# 1. LECTURA DE DATOS

In [35]:
# Importamos Librerías
import pandas as pd                                             # Se importa Pandas
from sklearn.neighbors import NearestNeighbors                  # Se importa NearestNeighbors de sklearn
from sklearn.preprocessing import MinMaxScaler                  # Se importa MinMaxScaler de sklearn
import numpy as np                                              # Se importa numpy
from sklearn.metrics import mean_squared_error                  # Se importa mean_squared_error de sklearn
from sklearn.model_selection import train_test_split            # Se importa train_test_split de sklearn

In [36]:
# Cargamos la data
data = pd.read_csv('movies_etl_eda.csv')                        # Cargamos la información dentro del dataframe data
data=data.sort_values('userId', ascending=True)                 # Se ordena de menor a mayor
df=data.head(100000)                                            # Cargamos 100 mil filas dentro de un dataframe más pequeño

# 2. PRE-PROCESAMIENTO

In [3]:
# Eliminamos duplicados
df_aux = df.drop_duplicates()                                   # Eliminar filas duplicadas
df=df_aux                                                       # Actualizamos el dataframe df

In [4]:
# Escalamos la data
scaler = MinMaxScaler()                                         # Utilizamos MinMaxScaler() para convertir la data entre 0 y 1
df['score'] = scaler.fit_transform(df[['score']])               # El escalamiento lo realizamos sobre la columna 'score'

In [5]:
# Transformamos la data en una matriz
user_movie_matrix = df.pivot_table(index='userId', columns='id', values='score')    # Creamos una matriz en la que cada fila represente un usuario y cada columna represente una película
user_movie_matrix = user_movie_matrix.fillna(0)                                     # Rellenamos los valores que faltan con ceros

# 3. VALIDACIÓN

In [7]:
# Dividimos los datos de calificaciones en conjuntos de entrenamiento y prueba
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [8]:
# Escalamos el score a un rango de 0 a 1
scaler = MinMaxScaler()                                                 # Habilitamos MinMaxScaler()
train_data['score'] = scaler.fit_transform(train_data[['score']])       # Escalamos train_data['score']
test_data['score'] = scaler.transform(test_data[['score']])             # Escalamos test_data['score']

In [9]:
# Creamos una matriz en la que cada fila represente un usuario y cada columna represente una película
train_matrix = train_data.pivot_table(index='userId', columns='id', values='score')     # Ingresamos train_data a una matriz
test_matrix = test_data.pivot_table(index='userId', columns='id', values='score')       # Ingresamos test_data a una matriz

In [10]:
# Reemplazamos los valores faltantes con ceros
train_matrix = train_matrix.fillna(0)                   # Reemplazamos los valores faltantes de train_matrix
test_matrix = test_matrix.fillna(0)                     # Reemplazamos los valores faltantes de test_matrix

In [11]:
# Entrenamos el modelo KNN
k = 5                                                                           # Número de vecinos a considerar
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k)     # Asignamos las variables al modelo
model.fit(train_matrix)                                                         # Entrenamos al modelo

NearestNeighbors(algorithm='brute', metric='cosine')

In [12]:
# Realizamos predicciones en el conjunto de prueba
test_indices = np.where(test_matrix.values.flatten() != 0)[0]                                                           # obtener los índices de valores distintos de cero en los datos de prueba
predictions = []                                                                                                        # inicializar una lista vacía para almacenar las calificaciones pronosticadas
for i in test_indices:                                                                                                  # bucle sobre los índices de datos de prueba distintos de cero
    user_index, movie_index = np.unravel_index(i, test_matrix.shape)                                                    # convertir el índice plano a los índices de fila y columna correspondientes en los datos de prueba
    distances, indices = model.kneighbors(train_matrix.iloc[user_index, :].values.reshape(1, -1), n_neighbors=k+1)      # encuentre los k usuarios más similares al usuario actual en función de sus calificaciones en los datos de entrenamiento
    rating = 0                                                                                                          # inicializar variables para la calificación predicha y la suma de los pesos (distancias)
    weight_sum = 0
    for j in range(1, len(distances.flatten())):                                                                        # recorrer los k usuarios más similares, omitiendo el primero (que es el usuario actual)
        similar_user_index = indices.flatten()[j]                                                                       # obtener el índice del j-ésimo usuario más similar en los datos de entrenamiento
        similar_user_rating = train_matrix.iloc[similar_user_index, movie_index]                                        # obtener la calificación del j-ésimo usuario más similar para la película actual
        similar_user_distance = distances.flatten()[j]                                                                  # obtener la distancia entre el usuario actual y el j-ésimo usuario más similar
        rating += similar_user_rating * similar_user_distance                                                           # actualice la calificación prevista y la suma de peso utilizando la calificación y la distancia del usuario más similar j-ésima
        weight_sum += similar_user_distance
    if weight_sum > 0:                                                                                                  # dividir la calificación pronosticada por la suma de los pesos para obtener el promedio ponderado
        rating /= weight_sum
    else:
        rating = 0
    predictions.append(rating)                                                                                          # agregar la calificación pronosticada a la lista de predicciones

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNe

In [13]:
# Calculamos el Root Mean Square Error RMSE
rmse = np.sqrt(mean_squared_error(test_matrix.values.flatten()[test_indices], predictions))         # Asignamos las matrices test_matrix y predictions
print(f'RMSE: {rmse:.4f}')                                                                          # Imprimimos el valor del RMSE

RMSE: 0.7330


# 4. EXPORTAR EL MODELO ENTRENADO

In [24]:
# Volvemos a correr el modelo KNN
k = 5                                                                           # Number of neighbors to consider
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k)     # Asignamos las variables al modelo
model.fit(user_movie_matrix)                                                    # Entrenamos al modelo

NearestNeighbors(algorithm='brute', metric='cosine')

In [37]:
# Exportamos el modelo
from joblib import dump                             # Importamos la libería joblib
dump(model, 'movies_modelo_entrenado.joblib')       # Exportamos el modelo entrenado

['movies_modelo_entrenado.joblib']

In [ ]:
# Importamos el modelo
from joblib import load

# Cargamos el modelo entrenado
model = load('movies_modelo_entrenado.joblib')

# 5. SOLUCIÓN

In [27]:
# Creamos las variables de prueba
user_id = 1                     # Creamos un id de usuario
movie_id = 'as1003'             # Creamos un id de película

In [34]:
# Hacer una recomendación para un usuario y una película

user_index = user_movie_matrix.index.get_loc(user_id)                                                               # Esta línea obtiene el índice de user_id en user_movie_matrix
movie_index = user_movie_matrix.columns.get_loc(movie_id)                                                           # Esta línea obtiene el índice de movie_id en user_movie_matrix
distances, indices = model.kneighbors(user_movie_matrix.iloc[user_index, :].values.reshape(1, -1), n_neighbors=k+1) # Clcular las distancias y los índices de los k+1 vecinos más cercanos a las calificaciones de películas del usuario.
recommended_movies = []                                                                                             # Esta línea inicializa una lista vacía que almacenará las películas recomendadas

# Este ciclo itera a través de los índices de los vecinos más cercanos y agrega las películas correspondientes a la lista de películas recomendadas.
for i in range(1, len(distances.flatten())):
    movie_index = user_movie_matrix.index[indices.flatten()[i]]
    recommended_movies.append(movie_index)

# Si el movie_id de entrada está en la lista de películas recomendadas, se recomienda al usuario
if movie_index in recommended_movies:
    s1 = user_movie_matrix.iloc[recommended_movies[0]].idxmax()                 # Recuperamos el id del primer vecino
    sp1 = data.loc[data['id'] == s1, 'title'].values[0]                         # Recuperamos el id y el título de la primera película 
    s2 = user_movie_matrix.iloc[recommended_movies[1]].idxmax()                 # Recuperamos el id del segundo vecino
    sp2 = data.loc[data['id'] == s2, 'title'].values[0]                         # Recuperamos el id y el título de la segunda película
    print(f'La pelicula {movie_id} es recomendada para el usuario {user_id}. También podrías ver: ','s1'," - ",sp1,". ",s2," - ",sp2,". ")

# Si el movie_id de entrada no está en la lista de películas recomendadas, entonces no se recomienda al usuario.
else:
    s1 = user_movie_matrix.iloc[recommended_movies[0]].idxmax()                 # Recuperamos el id del primer vecino
    sp1 = data.loc[data['id'] == s1, 'title'].values[0]                         # Recuperamos el id y el título de la primera película 
    s2 = user_movie_matrix.iloc[recommended_movies[1]].idxmax()                 # Recuperamos el id del segundo vecino
    sp2 = data.loc[data['id'] == s2, 'title'].values[0]                         # Recuperamos el id y el título de la segunda película 
    print(f'{movie_id} is not recommended for user {user_id}. Te recomendamos las siguientes películas: ',s1," - ",sp1,". ",s2," - ",sp2,". ")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


La pelicula as1003 es recomendada para el usuario 1. También podrías ver:  s1  -  princesses of the world .  as6759  -  thunderbird 6 . 
